#**ƒê·ªÅ t√†i: Ph√¢n t√≠ch, th·ªëng k√™ video th·ªãnh h√†nh tr√™n youtube theo c√°c qu·ªëc gia**


# 1. Gather data

## 1.1 Problem Statement
## **Tr·∫ßn Phan Minh Ti·∫øn**

    Question 1: V·∫Ω bi·ªÉu ƒë·ªì tr√≤n th·ªÉ hi·ªán t·ª∑ l·ªá s·ªë l∆∞·ª£ng video th·ªãnh h√†nh t·∫°i 10 qu·ªëc gia:
    Hoa K·ª≥ (US), Canada (CA), ƒê·ª©c (DE), Ph√°p (FR), Anh (GB), ·∫§n ƒê·ªô (IN), Nh·∫≠t B·∫£n (JP), H√†n Qu·ªëc (KR), Mexico (MX), Nga (RU).

    Question 2: V·∫Ω bi·ªÉu ƒë·ªì c·ªôt so s√°nh l∆∞·ª£t th√≠ch, kh√¥ng th√≠ch, l∆∞·ª£t xem v√† b√¨nh lu·∫≠n trung b√¨nh c·ªßa c√°c video th·ªãnh h√†nh.
##**Tr·∫ßn Minh Thu·∫≠n**

    Question 3: V·∫Ω bi·ªÉu ƒë·ªì heatmap (ho·∫∑c ma tr·∫≠n t∆∞∆°ng quan) th·ªÉ hi·ªán m·ª©c ƒë·ªô tr√πng l·∫∑p video th·ªãnh h√†nh gi·ªØa c√°c qu·ªëc gia.

    Question 4:  V·∫Ω bi·ªÉu ƒë·ªì ƒë∆∞·ªùng th·ªÉ hi·ªán t·ªïng l∆∞·ª£t xem video th·ªãnh h√†nh theo t·ª´ng th√°ng trong nƒÉm 2024.
##**Ho√†ng VƒÉn ƒê·ª©c**

    Question 5:  V·∫Ω bi·ªÉu ƒë·ªì c·ªôt nh√≥m th·ªÉ hi·ªán s·ªë l∆∞·ª£ng video th·ªãnh h√†nh theo qu√Ω trong c√°c nƒÉm 2015, 2020 v√† 2024.

    Question 6: V·∫Ω bi·ªÉu ƒë·ªì ƒë∆∞·ªùng ho·∫∑c c·ªôt th·ªÉ hi·ªán l∆∞·ª£t xem trung b√¨nh theo t·ª´ng ng√†y trong tu·∫ßn c·ªßa video th·ªãnh h√†nh nƒÉm 2024.

## 1.2 Import libraries:

In [35]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
from collections import Counter
import datetime
import wordcloud
import json


# T·∫¢I DATA XU·ªêNG T·ª™ KAGGLEHUB

In [36]:
# Download latest version
kaggle_download_path = kagglehub.dataset_download("datasnaek/youtube-new")
print("Path to dataset files:", kaggle_download_path)

Path to dataset files: /home/thuanc177/.cache/kagglehub/datasets/datasnaek/youtube-new/versions/115


In [37]:
# Hiding warnings for cleaner display
warnings.filterwarnings('ignore')

# Configuring some options
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# If you want interactive plots, uncomment the next line
# %matplotlib notebook

#G√°n M√£ m√†u, thu·ªôc t√≠nh cho ƒë·ªì th·ªã

In [38]:
PLOT_COLORS = ["#268bd2", "#0052CC", "#FF5722", "#b58900", "#003f5c"]
pd.options.display.float_format = '{:.2f}'.format
sns.set(style="ticks")
plt.rc('figure', figsize=(8, 5), dpi=100)
plt.rc('axes', labelpad=20, facecolor="#ffffff", linewidth=0.4, grid=True, labelsize=14)
plt.rc('patch', linewidth=0)
plt.rc('xtick.major', width=0.2)
plt.rc('ytick.major', width=0.2)
plt.rc('grid', color='#9E9E9E', linewidth=0.4)
plt.rc('font', family='DejaVu Sans', weight='400', size=10)
plt.rc('text', color='#282828')
plt.rc('savefig', pad_inches=0.3, dpi=300)

#ƒê·ªçc t·∫•t c·∫£ c√°c file

In [39]:
def read_and_fix_csv(path, fix_columns=None, default_encoding='utf-8', fallback_encoding='ISO-8859-1'):
    """
    ƒê·ªçc CSV v·ªõi ki·ªÉm tra encoding, v√† s·ª≠a l·ªói encoding ·ªü m·ªôt s·ªë c·ªôt n·∫øu c·∫ßn.
    """
    try:
        df = pd.read_csv(path, encoding=default_encoding)
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding=fallback_encoding)

    if fix_columns:
        for col in fix_columns:
            if col in df.columns:
                df[col] = df[col].apply(fix_encoding)

    return df

def fix_encoding(s):
    try:
        return s.encode('latin1').decode('utf-8')
    except:
        return s  # gi·ªØ nguy√™n n·∫øu kh√¥ng chuy·ªÉn ƒë∆∞·ª£c


In [40]:
# Danh s√°ch qu·ªëc gia v√† c√°c ƒë∆∞·ªùng d·∫´n t∆∞∆°ng ·ª©ng
country_codes = ['US', 'CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU']
columns_to_fix = ['title', 'channel_title', 'description', 'tags']

# T·∫°o dict l∆∞u c√°c DataFrame
dataframes = {}

for code in country_codes:
    filepath = f"{kaggle_download_path}/{code}videos.csv"
    fix = code in ['JP', 'KR', 'RU']  # JP, KR, RU c·∫ßn s·ª≠a l·ªói encoding
    df = read_and_fix_csv(filepath, fix_columns=columns_to_fix if fix else None)
    dataframes[code] = df

## 1.3 Xem th√¥ng tin c·ªßa Dataset

#VI·∫æT H√ÄM HI·ªÇN TH·ªä TH√îNG TIN C∆† B·∫¢N CHO T·ª™NG D·ªÆ LI·ªÜU C·ª¶A T·ª™NG QU·ªêC GIA

In [41]:
def display_basic_info(dataframes, country_names):
    """
    Hi·ªÉn th·ªã th√¥ng tin t·ªïng quan v√† chi ti·∫øt v·ªÅ d·ªØ li·ªáu t·ª´ng qu·ªëc gia.
    """
    print("\n=== üìä TH√îNG TIN C∆† B·∫¢N V·ªÄ D·ªÆ LI·ªÜU ===")

    total_records = 0
    for country, df in dataframes.items():
        print(f"\n{country_names.get(country, 'Unknown')} ({country}):")
        print(f"  - üìÑ S·ªë b·∫£n ghi: {len(df):,}")
        print(f"  - üß± S·ªë c·ªôt: {len(df.columns)}")
        print(f"  - üíæ K√≠ch th∆∞·ªõc b·ªô nh·ªõ: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

        # Video duy nh·∫•t
        unique_videos = df['video_id'].nunique() if 'video_id' in df.columns else 'N/A'
        print(f"  - üé• Video duy nh·∫•t: {unique_videos:,}")

        # Category ph·ªï bi·∫øn
        if 'category_id' in df.columns:
            top_categories = df['category_id'].value_counts().head(3).to_dict()
            print(f"  - üî¢ Top 3 category_id: {top_categories}")

        # M√¥ t·∫£ b·ªã thi·∫øu (NaN ho·∫∑c r·ªóng)
        missing_descriptions = df['description'].isna().sum() if 'description' in df.columns else 0
        empty_descriptions = (df['description'] == '').sum() if 'description' in df.columns else 0
        total_missing = missing_descriptions + empty_descriptions
        if 'description' in df.columns:
            missing_rate = total_missing / len(df) * 100
            print(f"  - üìù M√¥ t·∫£ thi·∫øu (NaN + r·ªóng): {total_missing:,} ({missing_rate:.2f}%)")

        # Ng√†y xu·∫•t b·∫£n s·ªõm nh·∫•t & mu·ªôn nh·∫•t
        if 'publish_time' in df.columns:
            try:
                df['publish_time'] = pd.to_datetime(df['publish_time'], errors='coerce')
                min_date = df['publish_time'].min()
                max_date = df['publish_time'].max()
                print(f"  - üìÜ Ng√†y ƒëƒÉng: {min_date.date()} ‚ûù {max_date.date()}")
            except Exception as e:
                print(f"  - ‚ö†Ô∏è L·ªói chuy·ªÉn ƒë·ªïi ng√†y: {e}")

        # C·ªôt c√≥ d·ªØ li·ªáu thi·∫øu
        null_columns = df.columns[df.isnull().any()]
        if len(null_columns):
            print(f"  - ‚ö†Ô∏è C√°c c·ªôt c√≥ missing values: {', '.join(null_columns)}")

        total_records += len(df)

    print(f"\nüìà T·ªîNG K·∫æT:")
    print(f"  - T·ªïng s·ªë b·∫£n ghi: {total_records:,}")
    print(f"  - S·ªë qu·ªëc gia: {len(dataframes)}")


#G√ÅN NH√ÉN T√äN CHO C√ÅC M√É QU·ªêC GIA

In [42]:
country_names = {
    'US': 'United States',
    'CA': 'Canada',
    'DE': 'Germany',
    'FR': 'France',
    'GB': 'United Kingdom',
    'IN': 'India',
    'JP': 'Japan',
    'KR': 'South Korea',
    'MX': 'Mexico',
    'RU': 'Russia'
}


#G·ªåI H√ÄM HI·ªÇN TH·ªä TH√îNG TIN D·ªÆ LI·ªÜU

In [43]:
display_basic_info(dataframes, country_names)


=== üìä TH√îNG TIN C∆† B·∫¢N V·ªÄ D·ªÆ LI·ªÜU ===

United States (US):
  - üìÑ S·ªë b·∫£n ghi: 40,949
  - üß± S·ªë c·ªôt: 16
  - üíæ K√≠ch th∆∞·ªõc b·ªô nh·ªõ: 109.6 MB
  - üé• Video duy nh·∫•t: 6,351
  - üî¢ Top 3 category_id: {24: 9964, 10: 6472, 26: 4146}
  - üìù M√¥ t·∫£ thi·∫øu (NaN + r·ªóng): 570 (1.39%)
  - üìÜ Ng√†y ƒëƒÉng: 2006-07-23 ‚ûù 2018-06-14
  - ‚ö†Ô∏è C√°c c·ªôt c√≥ missing values: description

Canada (CA):
  - üìÑ S·ªë b·∫£n ghi: 40,881
  - üß± S·ªë c·ªôt: 16
  - üíæ K√≠ch th∆∞·ªõc b·ªô nh·ªõ: 109.8 MB
  - üé• Video duy nh·∫•t: 24,427
  - üî¢ Top 3 category_id: {24: 13451, 25: 4159, 22: 4105}
  - üìù M√¥ t·∫£ thi·∫øu (NaN + r·ªóng): 1,296 (3.17%)
  - üìÜ Ng√†y ƒëƒÉng: 2008-01-13 ‚ûù 2018-06-14
  - ‚ö†Ô∏è C√°c c·ªôt c√≥ missing values: description

Germany (DE):
  - üìÑ S·ªë b·∫£n ghi: 40,840
  - üß± S·ªë c·ªôt: 16
  - üíæ K√≠ch th∆∞·ªõc b·ªô nh·ªõ: 114.2 MB
  - üé• Video duy nh·∫•t: 29,627
  - üî¢ Top 3 category_id: {24: 15292, 22: 5988, 25: 2935}

# 2. Truy c·∫≠p v√†o d·ªØ li·ªáu:

---



In [44]:
def basic_data_access(dataframes, country_names):
    print("\n" + "=" * 60)
    print("=== TRUY C·∫¨P V√Ä KH√ÅM PH√Å D·ªÆ LI·ªÜU C∆† B·∫¢N ===")
    print("=" * 60)

    for country, df in dataframes.items():
        print(f"\nüåç {country_names[country]} ({country})")
        print("-" * 50)

        print(f"üìä T·ªïng quan: {len(df):,} h√†ng | {len(df.columns)} c·ªôt | {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

        print("\nüìã C·∫•u tr√∫c d·ªØ li·ªáu:")
        for col in df.columns:
            print(f"   ‚Ä¢ {col:<20} | {str(df[col].dtype):<10} | Null: {df[col].isna().sum():,}")

        # 3. M·∫´u d·ªØ li·ªáu
        print("\nüëÄ M·∫´u d·ªØ li·ªáu (5 d√≤ng):")
        try:
            display_cols = ['title', 'channel_title', 'views', 'likes', 'trending_date']
            print(df[display_cols].head().to_string(index=False, max_colwidth=30))
        except:
            print("   ‚ö† Kh√¥ng th·ªÉ hi·ªÉn th·ªã m·∫´u")

        # 4. Th·ªëng k√™ m√¥ t·∫£
        print("\nüìà Th·ªëng k√™ m√¥ t·∫£:")
        num_cols = ['views', 'likes', 'dislikes', 'comment_count']
        num_cols = [col for col in num_cols if col in df.columns]
        if num_cols:
            print(df[num_cols].describe().round(1).to_string())
        else:
            print("   ‚ö† Kh√¥ng c√≥ c·ªôt s·ªë")

        # 5. Missing values
        print("\n‚ùì Missing values:")
        missing = df.isna().sum()
        missing = missing[missing > 0]
        if not missing.empty:
            for col, val in missing.items():
                print(f"   ‚Ä¢ {col}: {val:,} ({val / len(df) * 100:.1f}%)")
        else:
            print("   ‚úì Kh√¥ng c√≥ missing values")

        # 6. Top categories
        print("\nüèÜ Top 5 categories:")
        if 'category_id' in df.columns:
            print(df['category_id'].value_counts().head().to_string())
        else:
            print("   ‚ö† Kh√¥ng c√≥ c·ªôt category_id")

        # 7. Top channels
        print("\nüé¨ Top 5 channels:")
        if 'channel_title' in df.columns:
            print(df['channel_title'].value_counts().head().to_string())
        else:
            print("   ‚ö† Kh√¥ng c√≥ c·ªôt channel_title")

        # 8. Ph√¢n t√≠ch th·ªùi gian
        print("\nüìÖ Ph√¢n t√≠ch th·ªùi gian:")
        if 'trending_date' in df.columns:
            try:
                dates = pd.to_datetime(df['trending_date'], errors='coerce')
                print(f"   ‚Ä¢ {dates.min().date()} ‚Üí {dates.max().date()} ({(dates.max() - dates.min()).days} ng√†y)")
                print("   ‚Ä¢ Theo nƒÉm:")
                print(dates.dt.year.value_counts().sort_index().to_string())
            except:
                print("   ‚ö† Kh√¥ng th·ªÉ ph√¢n t√≠ch trending_date")
        else:
            print("   ‚ö† Kh√¥ng c√≥ c·ªôt trending_date")

        # 9. Top performers
        print("\nüåü Top performers:")
        for col in ['views', 'likes', 'comment_count']:
            if col in df.columns:
                idx = df[col].idxmax()
                title = df.at[idx, 'title'] if 'title' in df.columns else 'Kh√¥ng r√µ'
                print(f"   ‚Ä¢ Max {col}: {df.at[idx, col]:,} - '{title[:40]}...'")

        # 10. Ki·ªÉm tra ch·∫•t l∆∞·ª£ng
        print("\nüîç Ki·ªÉm tra ch·∫•t l∆∞·ª£ng:")
        issues = []
        for col in ['views', 'likes', 'dislikes']:
            if col in df.columns and (df[col] < 0).any():
                issues.append(f"{col} √¢m: {(df[col] < 0).sum()}")

        if 'title' in df.columns:
            empty_titles = df['title'].str.strip().eq('').sum()
            if empty_titles > 0:
                issues.append(f"Title r·ªóng: {empty_titles}")

        if 'video_id' in df.columns:
            dup = df.duplicated('video_id').sum()
            if dup > 0:
                issues.append(f"Tr√πng video_id: {dup}")

        if issues:
            for issue in issues:
                print(f"   ‚ö† {issue}")
        else:
            print("   ‚úì Kh√¥ng ph√°t hi·ªán v·∫•n ƒë·ªÅ r√µ r√†ng")

        print("=" * 50)


In [45]:
basic_data_access(dataframes, country_names)


=== TRUY C·∫¨P V√Ä KH√ÅM PH√Å D·ªÆ LI·ªÜU C∆† B·∫¢N ===

üåç United States (US)
--------------------------------------------------
üìä T·ªïng quan: 40,949 h√†ng | 16 c·ªôt | 107.03 MB

üìã C·∫•u tr√∫c d·ªØ li·ªáu:
   ‚Ä¢ video_id             | object     | Null: 0
   ‚Ä¢ trending_date        | object     | Null: 0
   ‚Ä¢ title                | object     | Null: 0
   ‚Ä¢ channel_title        | object     | Null: 0
   ‚Ä¢ category_id          | int64      | Null: 0
   ‚Ä¢ publish_time         | datetime64[ns, UTC] | Null: 0
   ‚Ä¢ tags                 | object     | Null: 0
   ‚Ä¢ views                | int64      | Null: 0
   ‚Ä¢ likes                | int64      | Null: 0
   ‚Ä¢ dislikes             | int64      | Null: 0
   ‚Ä¢ comment_count        | int64      | Null: 0
   ‚Ä¢ thumbnail_link       | object     | Null: 0
   ‚Ä¢ comments_disabled    | bool       | Null: 0
   ‚Ä¢ ratings_disabled     | bool       | Null: 0
   ‚Ä¢ video_error_or_removed | bool       | Null: 0
   ‚Ä¢

## 2.1 Th√™m c·ªôt

# Th√™m c√°c c·ªôt ph√¢n t√≠ch hi·ªáu su·∫•t v√†o DataFrame.

In [46]:
def add_performance_metrics(df):
    """
    Th√™m c√°c c·ªôt ph√¢n t√≠ch hi·ªáu su·∫•t v√†o DataFrame.

    Th√™m c√°c c·ªôt:
        - engagement_rate (%)
        - like_dislike_ratio
        - viral_score
        - performance_category

    Tr·∫£ v·ªÅ: DataFrame m·ªõi v·ªõi c√°c c·ªôt b·ªï sung.
    """
    if df is None or df.empty:
        print("‚ö†Ô∏è DataFrame r·ªóng ho·∫∑c kh√¥ng h·ª£p l·ªá.")
        return df

    print("üöÄ Th√™m c·ªôt ph√¢n t√≠ch hi·ªáu su·∫•t...")

    df = df.copy()

    # Engagement Rate (%)
    df['engagement_rate'] = ((df['likes'] + df['comment_count']) / df['views'].replace(0, np.nan) * 100).round(2)
    df['engagement_rate'] = df['engagement_rate'].fillna(0).clip(lower=0)

    # Like/Dislike Ratio
    df['like_dislike_ratio'] = np.where(
        df['dislikes'] > 0,
        (df['likes'] / df['dislikes']).round(2),
        np.where(df['likes'] > 0, np.inf, 0)
    )

    # Viral Score
    max_views = df['views'].max() or 1
    max_engagement = df['engagement_rate'].max() or 1

    df['viral_score'] = (
        (df['views'] / max_views * 0.7 +
         df['engagement_rate'] / max_engagement * 0.3) * 100
    ).round(2)

    # Performance Category
    df['performance_category'] = pd.cut(
        df['viral_score'],
        bins=[0, 20, 40, 60, 80, 100],
        labels=['Low', 'Below Average', 'Average', 'Good', 'Excellent'],
        include_lowest=True
    )

    print("‚úÖ ƒê√£ th√™m: engagement_rate, like_dislike_ratio, viral_score, performance_category")
    return df

In [47]:
def add_time_analysis(df):
    """
    Th√™m c√°c c·ªôt ph√¢n t√≠ch th·ªùi gian v√†o DataFrame.
    """
    print("‚è∞ Th√™m c·ªôt ph√¢n t√≠ch th·ªùi gian...")

    df = df.copy()
    df['trending_datetime'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m', errors='coerce')

    df['trending_weekday'] = df['trending_datetime'].dt.day_name()
    df['trending_month'] = df['trending_datetime'].dt.month_name()
    df['is_weekend_trending'] = df['trending_datetime'].dt.weekday >= 5
    df['trending_week'] = df['trending_datetime'].dt.isocalendar().week

    if 'publish_time' in df.columns:
        df['publish_datetime'] = pd.to_datetime(df['publish_time'], errors='coerce')
        df['upload_to_trending_days'] = (df['trending_datetime'] - df['publish_datetime']).dt.days

    print("‚úÖ ƒê√£ th√™m: trending_weekday, trending_month, is_weekend_trending, trending_week, upload_to_trending_days")
    return df

In [48]:
def predict_content_type(title):
    """
    D·ª± ƒëo√°n lo·∫°i n·ªôi dung d·ª±a tr√™n ti√™u ƒë·ªÅ
    """
    title_lower = str(title).lower()

    music_keywords = ['official', 'mv', 'music video', 'song', 'audio', 'feat', 'ft']
    tutorial_keywords = ['how to', 'tutorial', 'guide', 'tips', 'diy', 'review']
    news_keywords = ['news', 'breaking', 'report', 'update', 'interview']
    entertainment_keywords = ['funny', 'comedy', 'reaction', 'prank', 'challenge']
    gaming_keywords = ['gameplay', 'gaming', 'let\'s play', 'walkthrough']

    if any(keyword in title_lower for keyword in music_keywords):
        return 'Music'
    elif any(keyword in title_lower for keyword in tutorial_keywords):
        return 'Tutorial'
    elif any(keyword in title_lower for keyword in news_keywords):
        return 'News'
    elif any(keyword in title_lower for keyword in entertainment_keywords):
        return 'Entertainment'
    elif any(keyword in title_lower for keyword in gaming_keywords):
        return 'Gaming'
    else:
        return 'Other'


##**Nh·∫≠n x√©t d·ªØ li·ªáu:**
-
-
-

# 3. Clean data:

#LO·∫†I B·ªé B·∫¢N GHI TR√ôNG L·∫∂P

In [49]:
def _remove_duplicates(self, df):
        """
        Lo·∫°i b·ªè duplicates
        """
        initial_count = len(df)

        # Lo·∫°i b·ªè duplicate d·ª±a tr√™n video_id v√† trending_date
        df = df.drop_duplicates(subset=['video_id', 'trending_date'], keep='first')

        removed_count = initial_count - len(df)
        if removed_count > 0:
            print(f"  ‚úì ƒê√£ lo·∫°i b·ªè {removed_count:,} b·∫£n ghi tr√πng l·∫∑p")

        return df

#X·ª¨ L√ù MISSING VALUES

In [50]:
def _handle_missing_values(self, df):
        """
        X·ª≠ l√Ω missing values
        """
        # Fill missing boolean values
        bool_cols = ['comments_disabled', 'ratings_disabled', 'video_error_or_removed']
        for col in bool_cols:
            df[col] = df[col].fillna(False)

        # Fill missing category_id v·ªõi mode
        if df['category_id'].isna().sum() > 0:
            mode_category = df['category_id'].mode().iloc[0] if not df['category_id'].mode().empty else 0
            df['category_id'] = df['category_id'].fillna(mode_category)

        return df

#L√ÄM S·∫†CH C√ÅC C·ªòT C√ì D·∫†NG D·ªÆ LI·ªÜU TEXT

In [51]:
def clean_text_columns(df):
        """
        L√†m s·∫°ch c√°c c·ªôt text
        """
        # L√†m s·∫°ch title
        df['title'] = df['title'].fillna('').astype(str)
        df['title'] = df['title'].str.strip()

        # L√†m s·∫°ch channel_title
        df['channel_title'] = df['channel_title'].fillna('').astype(str)
        df['channel_title'] = df['channel_title'].str.strip()

        # L√†m s·∫°ch tags
        df['tags'] = df['tags'].fillna('').astype(str)
        df['tags'] = df['tags'].str.replace('[none]', '', regex=False)

        # ƒê·∫øm s·ªë l∆∞·ª£ng tags
        df['tag_count'] = df['tags'].apply(lambda x: len([tag.strip() for tag in x.split('|') if tag.strip()]) if x else 0)

        # L√†m s·∫°ch description
        df['description'] = df['description'].fillna('').astype(str)
        df['description_length'] = df['description'].str.len()

        # T·∫°o title length
        df['title_length'] = df['title'].str.len()

        return df


#L√ÄM S·∫†CH C√ÅC C·ªòT C√ì D·∫†NG D·ªÆ LI·ªÜU S·ªê

In [52]:
def clean_numeric_columns(df):
        """
        L√†m s·∫°ch c√°c c·ªôt s·ªë
        """
        numeric_cols = ['views', 'likes', 'dislikes', 'comment_count']

        for col in numeric_cols:
            # Chuy·ªÉn ƒë·ªïi v·ªÅ s·ªë v√† x·ª≠ l√Ω gi√° tr·ªã √¢m
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0).astype(int)
            df[col] = df[col].abs()  # Lo·∫°i b·ªè gi√° tr·ªã √¢m

        # T·∫°o c√°c metric t·ª∑ l·ªá
        df['like_rate'] = df['likes'] / (df['views'] + 1)  # +1 ƒë·ªÉ tr√°nh chia cho 0
        df['dislike_rate'] = df['dislikes'] / (df['views'] + 1)
        df['comment_rate'] = df['comment_count'] / (df['views'] + 1)
        df['engagement_rate'] = (df['likes'] + df['dislikes'] + df['comment_count']) / (df['views'] + 1)

        return df

#L√ÄM S·∫†CH C√ÅC C·ªòT CH·ª®A D·∫†NG D·ªÆ LI·ªÜU DATETIME

In [53]:
def clean_datetime_columns(df):
    if 'publish_time' in df.columns:
        df['publish_time'] = pd.to_datetime(df['publish_time'], errors='coerce')
    return df

def clean_numeric_columns(df):
    numeric_cols = ['views', 'likes', 'dislikes', 'comment_count']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def clean_text_columns(df):
    text_cols = ['title', 'description', 'tags', 'channel_title']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()
    return df

def handle_missing_values(df):
    return df.dropna(subset=['title', 'video_id'], how='any') if 'video_id' in df.columns else df

def remove_duplicates(df):
    return df.drop_duplicates(subset=['video_id', 'publish_time']) if 'video_id' in df.columns else df


#H√ÄM L√ÄM S·∫†CH CH√çNH

In [54]:
def clean_data(dataframes, country_names):
    """
    L√†m s·∫°ch d·ªØ li·ªáu cho t·∫•t c·∫£ c√°c qu·ªëc gia (kh√¥ng d√πng class).
    """
    print("\n=== üßπ B·∫ÆT ƒê·∫¶U L√ÄM S·∫†CH D·ªÆ LI·ªÜU ===")

    for country, df in dataframes.items():
        print(f"\nüîß ƒêang x·ª≠ l√Ω {country_names.get(country, 'Unknown')} ({country})...")
        original_rows = len(df)

        try:
            # 1. X·ª≠ l√Ω th·ªùi gian
            df = clean_datetime_columns(df)
            print("  ‚úì ƒê√£ x·ª≠ l√Ω th·ªùi gian")

            # 2. Chu·∫©n ho√° s·ªë li·ªáu
            df = clean_numeric_columns(df)
            print("  ‚úì ƒê√£ chu·∫©n ho√° s·ªë li·ªáu")

            # 3. L√†m s·∫°ch text
            df = clean_text_columns(df)
            print("  ‚úì ƒê√£ l√†m s·∫°ch text")

            # 4. X·ª≠ l√Ω missing values
            df = handle_missing_values(df)
            print("  ‚úì ƒê√£ x·ª≠ l√Ω missing values")

            # 5. Xo√° tr√πng l·∫∑p
            df = remove_duplicates(df)
            print("  ‚úì ƒê√£ lo·∫°i b·ªè b·∫£n ghi tr√πng l·∫∑p")

            # 6. Th√™m th√¥ng tin qu·ªëc gia
            df['country'] = country
            df['country_name'] = country_names[country]

            cleaned_rows = len(df)
            removed = original_rows - cleaned_rows
            print(f"  ‚Üí T·ªïng b·∫£n ghi sau l√†m s·∫°ch: {cleaned_rows:,} (gi·∫£m {removed:,})")

            # C·∫≠p nh·∫≠t l·∫°i
            dataframes[country] = df

        except Exception as e:
            print(f"‚ùå L·ªói khi x·ª≠ l√Ω {country}: {e}")

    print("\n‚úÖ Ho√†n t·∫•t l√†m s·∫°ch d·ªØ li·ªáu cho t·∫•t c·∫£ qu·ªëc gia.")
    return dataframes


In [55]:
dataframes = clean_data(dataframes, country_names)


=== üßπ B·∫ÆT ƒê·∫¶U L√ÄM S·∫†CH D·ªÆ LI·ªÜU ===

üîß ƒêang x·ª≠ l√Ω United States (US)...
  ‚úì ƒê√£ x·ª≠ l√Ω th·ªùi gian
  ‚úì ƒê√£ chu·∫©n ho√° s·ªë li·ªáu
  ‚úì ƒê√£ l√†m s·∫°ch text
  ‚úì ƒê√£ x·ª≠ l√Ω missing values
  ‚úì ƒê√£ lo·∫°i b·ªè b·∫£n ghi tr√πng l·∫∑p
  ‚Üí T·ªïng b·∫£n ghi sau l√†m s·∫°ch: 6,354 (gi·∫£m 34,595)

üîß ƒêang x·ª≠ l√Ω Canada (CA)...
  ‚úì ƒê√£ x·ª≠ l√Ω th·ªùi gian
  ‚úì ƒê√£ chu·∫©n ho√° s·ªë li·ªáu
  ‚úì ƒê√£ l√†m s·∫°ch text
  ‚úì ƒê√£ x·ª≠ l√Ω missing values
  ‚úì ƒê√£ lo·∫°i b·ªè b·∫£n ghi tr√πng l·∫∑p
  ‚Üí T·ªïng b·∫£n ghi sau l√†m s·∫°ch: 24,433 (gi·∫£m 16,448)

üîß ƒêang x·ª≠ l√Ω Germany (DE)...
  ‚úì ƒê√£ x·ª≠ l√Ω th·ªùi gian
  ‚úì ƒê√£ chu·∫©n ho√° s·ªë li·ªáu
  ‚úì ƒê√£ l√†m s·∫°ch text
  ‚úì ƒê√£ x·ª≠ l√Ω missing values
  ‚úì ƒê√£ lo·∫°i b·ªè b·∫£n ghi tr√πng l·∫∑p
  ‚Üí T·ªïng b·∫£n ghi sau l√†m s·∫°ch: 29,632 (gi·∫£m 11,208)

üîß ƒêang x·ª≠ l√Ω France (FR)...
  ‚úì ƒê√£ x·ª≠ l√Ω th·ªùi gian
  ‚úì ƒê√£ chu·∫©n ho√° s·ªë li·ªáu
  ‚úì ƒê√£

#T·∫†O DATASET T·ªîNG H·ª¢P (MERGE)

In [56]:
def create_combined_dataset(dataframes):
    """
    T·∫°o dataset t·ªïng h·ª£p t·ª´ t·∫•t c·∫£ c√°c qu·ªëc gia.
    """
    print("\n=== üîÑ T·∫†O DATASET T·ªîNG H·ª¢P ===")

    if not dataframes:
        print("‚úó Kh√¥ng c√≥ d·ªØ li·ªáu ƒë·ªÉ k·∫øt h·ª£p")
        return None

    # K·∫øt h·ª£p t·∫•t c·∫£ DataFrame
    combined_df = pd.concat(dataframes.values(), ignore_index=True)

    print(f"‚úì ƒê√£ t·∫°o dataset t·ªïng h·ª£p v·ªõi:")
    print(f"  - T·ªïng s·ªë b·∫£n ghi: {len(combined_df):,}")
    print(f"  - S·ªë c·ªôt: {len(combined_df.columns)}")
    print(f"  - S·ªë qu·ªëc gia: {len(dataframes)}")
    print(f"  - K√≠ch th∆∞·ªõc b·ªô nh·ªõ: {combined_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    return combined_df


In [57]:
combined_df = create_combined_dataset(dataframes)


=== üîÑ T·∫†O DATASET T·ªîNG H·ª¢P ===
‚úì ƒê√£ t·∫°o dataset t·ªïng h·ª£p v·ªõi:
  - T·ªïng s·ªë b·∫£n ghi: 208,718
  - S·ªë c·ªôt: 18
  - S·ªë qu·ªëc gia: 10
  - K√≠ch th∆∞·ªõc b·ªô nh·ªõ: 488.90 MB


#**4.Save new after clean**

##4.1. REPORT CH·∫§T L∆Ø·ª¢NG C·ª¶A D·ªÆ LI·ªÜU

In [58]:
from datetime import datetime

def generate_data_quality_report(dataframes, country_names):
    """
    T·∫°o b√°o c√°o ch·∫•t l∆∞·ª£ng d·ªØ li·ªáu cho t·ª´ng qu·ªëc gia.
    """
    print("\n=== üìã B√ÅO C√ÅO CH·∫§T L∆Ø·ª¢NG D·ªÆ LI·ªÜU ===")

    for country, df in dataframes.items():
        print(f"\n{country_names[country]} ({country}):")

        # 1. Missing values
        missing_counts = df.isnull().sum()
        if missing_counts.sum() > 0:
            print("  ‚ùó Missing values:")
            for col, count in missing_counts[missing_counts > 0].items():
                print(f"    - {col}: {count:,} ({count/len(df)*100:.1f}%)")
        else:
            print("  ‚úì Kh√¥ng c√≥ missing values")

        # 2. Ki·ªÉm tra ch·∫•t l∆∞·ª£ng d·ªØ li·ªáu
        print("  üîç Ki·ªÉm tra ch·∫•t l∆∞·ª£ng:")

        # Gi√° tr·ªã √¢m
        negatives = {}
        for col in ['views', 'likes', 'dislikes', 'comment_count']:
            if col in df.columns:
                negatives[col] = (df[col] < 0).sum()

        if all(v == 0 for v in negatives.values()):
            print("    ‚úì Kh√¥ng c√≥ gi√° tr·ªã s·ªë √¢m")
        else:
            for col, count in negatives.items():
                if count > 0:
                    print(f"    - {col}: {count:,} gi√° tr·ªã √¢m")

        # Ng√†y trong t∆∞∆°ng lai (trending_date)
        if 'trending_date' in df.columns:
            try:
                df['trending_date'] = pd.to_datetime(df['trending_date'], errors='coerce')
                future_dates = (df['trending_date'] > datetime.now()).sum()
                if future_dates == 0:
                    print("    ‚úì Kh√¥ng c√≥ ng√†y trending trong t∆∞∆°ng lai")
                else:
                    print(f"    - {future_dates:,} b·∫£n ghi c√≥ ng√†y trending trong t∆∞∆°ng lai")
            except:
                print("    ‚ö† Kh√¥ng th·ªÉ ph√¢n t√≠ch trending_date")

        # Title tr·ªëng
        if 'title' in df.columns:
            empty_titles = df['title'].isna().sum() + (df['title'].str.strip() == '').sum()
            if empty_titles == 0:
                print("    ‚úì T·∫•t c·∫£ video ƒë·ªÅu c√≥ ti√™u ƒë·ªÅ")
            else:
                print(f"    - {empty_titles:,} video kh√¥ng c√≥ ti√™u ƒë·ªÅ")

        # Description tr·ªëng
        if 'description' in df.columns:
            empty_desc = df['description'].isna().sum() + (df['description'].str.strip() == '').sum()
            if empty_desc == 0:
                print("    ‚úì T·∫•t c·∫£ video ƒë·ªÅu c√≥ m√¥ t·∫£")
            else:
                print(f"    - {empty_desc:,} video kh√¥ng c√≥ m√¥ t·∫£")


In [59]:
generate_data_quality_report(dataframes, country_names)


=== üìã B√ÅO C√ÅO CH·∫§T L∆Ø·ª¢NG D·ªÆ LI·ªÜU ===

United States (US):
  ‚úì Kh√¥ng c√≥ missing values
  üîç Ki·ªÉm tra ch·∫•t l∆∞·ª£ng:
    ‚úì Kh√¥ng c√≥ gi√° tr·ªã s·ªë √¢m
    ‚úì Kh√¥ng c√≥ ng√†y trending trong t∆∞∆°ng lai
    ‚úì T·∫•t c·∫£ video ƒë·ªÅu c√≥ ti√™u ƒë·ªÅ
    - 4 video kh√¥ng c√≥ m√¥ t·∫£

Canada (CA):
  ‚úì Kh√¥ng c√≥ missing values
  üîç Ki·ªÉm tra ch·∫•t l∆∞·ª£ng:
    ‚úì Kh√¥ng c√≥ gi√° tr·ªã s·ªë √¢m
    ‚úì Kh√¥ng c√≥ ng√†y trending trong t∆∞∆°ng lai
    ‚úì T·∫•t c·∫£ video ƒë·ªÅu c√≥ ti√™u ƒë·ªÅ
    ‚úì T·∫•t c·∫£ video ƒë·ªÅu c√≥ m√¥ t·∫£

Germany (DE):
  ‚úì Kh√¥ng c√≥ missing values
  üîç Ki·ªÉm tra ch·∫•t l∆∞·ª£ng:
    ‚úì Kh√¥ng c√≥ gi√° tr·ªã s·ªë √¢m
    ‚úì Kh√¥ng c√≥ ng√†y trending trong t∆∞∆°ng lai
    ‚úì T·∫•t c·∫£ video ƒë·ªÅu c√≥ ti√™u ƒë·ªÅ
    ‚úì T·∫•t c·∫£ video ƒë·ªÅu c√≥ m√¥ t·∫£

France (FR):
  ‚úì Kh√¥ng c√≥ missing values
  üîç Ki·ªÉm tra ch·∫•t l∆∞·ª£ng:
    ‚úì Kh√¥ng c√≥ gi√° tr·ªã s·ªë √¢m
    ‚úì Kh√¥ng c√≥ ng√†y trending tr

In [60]:
# Hi·ªÉn th·ªã 10 m·∫´u d·ªØ li·ªáu t·ª´ dataset t·ªïng h·ª£p
combined_df.sample(10, random_state=42)

video_id trending_date  \
60986   KstndSY3_-s      17.17.11   
147939  fLA-GODL6oo      17.31.12   
158868  amPhdzn_zYs      18.08.03   
174164  wjpcemcfYGU      17.15.11   
115368  rHHmejZNs_Y      18.23.03   
21317   oc6FyKbURBw      18.24.03   
18213   W8f8CXJ8ReA      18.26.02   
79783   gWo3WD-saCY      18.26.03   
186002  NySvsRog5vQ      18.25.01   
82534   1s-h-bx-cbY      18.19.04   

                                                    title  \
60986   A Boatload of Fails: Throwback Fails (November...   
147939  ANA LAGO SE PONE FURIOSA CUANDO ANTONIETA LE H...   
158868  Tanda de penales de Rayados 1 (2) - (3) 1 Quer...   
174164  –ò–º–º–µ—Ä—Å–∏–≤–Ω—ã–π –æ–±–∑–æ—Ä iPhone X - 14 –Ω–æ—è–±—Ä—è –≤ 20:00...   
115368  Ê∞¥ÊõúÊó•„ÅÆ„ÉÄ„Ç¶„É≥„Çø„Ç¶„É≥ 2018Âπ¥3Êúà21Êó• SASUKE„ÅÆÊ±†„ÅÆÊ∞¥„Åú„Çì„Å∂Êäú„Åè SASUKE„ÅÆÂ§±...   
21317   Shields and Brooks on John Bolton‚Äôs worldview,...   
18213                                 I'M TOO COMPETITIVE   
79783   Les meilleurs moments de Pascal le grand fr√®re...   
186002                 10 –û–®–ò–ë–û–ö –ü–†–ò –ü–ï–†–ï–ü–ò–°–ö–ï –° –î–ï–í–£–®–ö–û–ô   
82534   Sean Hannity Had Exclusively Almost Nothing to...   

                          channel_title  category_id  \
60986                          FailArmy           23   
147939                     Carlos Cohen           24   
158868                Televisa Deportes           17   
174164                         Wylsacom           28   
115368                      PushinaMito           23   
21317                      PBS NewsHour           25   
18213           Young Don The Sauce God           23   
79783                     FIFOU Best Of           24   
186002                          tophype           24   
82534   The Daily Show with Trevor Noah           23   

                    publish_time  \
60986  2017-11-16 14:30:01+00:00   
147939 2017-12-30 09:27:53+00:00   
158868 2018-03-08 03:09:47+00:00   
174164 2017-11-14 20:50:11+00:00   
115368 2018-03-21 18:34:31+00:00   
21317  2018-03-23 22:55:42+00:00   
18213  2018-02-25 17:02:53+00:00   
79783  2018-03-05 14:49:47+00:00   
186002 2018-01-24 08:10:36+00:00   
82534  2018-04-18 03:30:00+00:00   

                                                     tags    views  likes  \
60986   failarmy"|"fail army"|"fails"|"fail laugh"|"fu...   736556  14610   
147939  Estaci√É¬≥n Arcade|"Carlos Cohen"|"exatlon"|"mex...    83882    260   
158868  Televisa|"Televisa deportes"|"futbol"|"program...    89928    506   
174164  iPhone|"Apple"|"wylsa"|"Wylsacom"|"–∞–π—Ñ–æ–Ω"|"iPh...   886496  70814   
115368                                             [none]   417775    950   
21317                                              [none]    60237    731   
18213   young don the sauce god|"animations"|"animated...   392727  26539   
79783   pascal le grand fr√®re"|"Pascal le grand fr√®re"...    23813    212   
186002  —Ç–æ–ø|"—Ç–æ–ø 5"|"—Ç–æ–ø10"|"—Ç–æ–ø5"|"top 10"|"tophype"|...   225747   6374   
82534   the daily show"|"trevor noah"|"daily show with...  1657668  23458   

        dislikes  comment_count  \
60986        852            708   
147939       122            148   
158868        47            229   
174164      4486            421   
115368       415            385   
21317         36              0   
18213        267           3132   
79783         25             69   
186002       872           3253   
82534        783           1881   

                                        thumbnail_link  comments_disabled  \
60986   https://i.ytimg.com/vi/KstndSY3_-s/default.jpg              False   
147939  https://i.ytimg.com/vi/fLA-GODL6oo/default.jpg              False   
158868  https://i.ytimg.com/vi/amPhdzn_zYs/default.jpg              False   
174164  https://i.ytimg.com/vi/wjpcemcfYGU/default.jpg              False   
115368  https://i.ytimg.com/vi/rHHmejZNs_Y/default.jpg              False   
21317   https://i.ytimg.com/vi/oc6FyKbURBw/default.jpg               True   
18213   https://i.ytim

# **5.Answer the research question (Visuazation)**

##**Tr·∫ßn Phan Minh Ti·∫øn**
### **Question 1**: V·∫Ω bi·ªÉu ƒë·ªì tr√≤n th·ªÉ hi·ªán t·ª∑ l·ªá s·ªë l∆∞·ª£ng video th·ªãnh h√†nh t·∫°i 10 qu·ªëc gia: Hoa K·ª≥ (US), Canada (CA), ƒê·ª©c (DE), Ph√°p (FR), Anh (GB), ·∫§n ƒê·ªô (IN), Nh·∫≠t B·∫£n (JP), H√†n Qu·ªëc (KR), Mexico (MX), Nga (RU).


##**Tr·∫ßn Phan Minh Ti·∫øn**
### **Question 2**: V·∫Ω bi·ªÉu ƒë·ªì c·ªôt so s√°nh l∆∞·ª£t th√≠ch, kh√¥ng th√≠ch, l∆∞·ª£t xem v√† b√¨nh lu·∫≠n trung b√¨nh c·ªßa c√°c video th·ªãnh h√†nh.

##**Ho√†ng VƒÉn ƒê·ª©c**

    Question 5:  V·∫Ω bi·ªÉu ƒë·ªì c·ªôt nh√≥m th·ªÉ hi·ªán s·ªë l∆∞·ª£ng video th·ªãnh h√†nh theo qu√Ω trong c√°c nƒÉm 2015, 2020 v√† 2024.
